In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
os.listdir('/kaggle/input/dogs-vs-cats')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# library for reading and showing images
import matplotlib.image as mpimg

# importing all the required sub-modules from keras
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

from keras.preprocessing.image import img_to_array, load_img
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Input,Dropout,GlobalAveragePooling2D

In [ ]:
import zipfile
with zipfile.ZipFile('../input/dogs-vs-cats/test1.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
with zipfile.ZipFile('../input/dogs-vs-cats/train.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
## Alphabetical 

In [ ]:
y_datset = []
x_datset = []
min_dim = 0
for file in os.listdir('./train'):
    x_datset.append(file)   
    if file.startswith('cat'):
        y_datset.append('0')
    else:
        y_datset.append('1')

train_df = pd.DataFrame({'filename': x_datset, 'class': y_datset})
print(train_df.head())

In [ ]:
x_train, x_val= train_test_split(train_df, test_size=0.33, random_state=42)
x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)

In [ ]:
valImg = plt.imread('./train/'+x_val.iloc[0,0])
#print(x_testImg[0])
plt.imshow(valImg)
plt.show()


In [ ]:
trainImg = plt.imread('./train/'+x_train.iloc[0,0])
plt.imshow(trainImg)
plt.show()


In [ ]:
x_val.head()

In [ ]:
# create two generators to augment/scale input data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# create batches of data
dir_train = './train'
x_col = 'filename'
y_col='class'
class_mode = 'binary'
target_size=(224,224)
train_generator = train_datagen.flow_from_dataframe(x_train, directory=dir_train, x_col=x_col,
                                                    y_col=y_col, class_mode=class_mode, 
                                                    target_size=target_size)
val_generator = val_datagen.flow_from_dataframe(x_val, directory=dir_train, x_col=x_col, 
                                                  y_col=y_col, class_mode=class_mode, 
                                                  target_size=target_size)

#### Another method of train/val split -- without using sklearn train_test split

In [ ]:
train_df.head()

In [ ]:
# create two generators to augment/scale input data
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.33,
                                   horizontal_flip=True)

# create batches of data
dir_train = './train'
x_col = 'filename'
y_col='class'
class_mode = 'binary'
target_size=(224,224)
train_generator = train_datagen.flow_from_dataframe(train_df, directory=dir_train, x_col=x_col,
                                                    y_col=y_col, class_mode=class_mode, 
                                                    target_size=target_size,subset='training')
val_generator = train_datagen.flow_from_dataframe(train_df, directory=dir_train, x_col=x_col, 
                                                  y_col=y_col, class_mode=class_mode, 
                                                  target_size=target_size,subset='validation')

In [ ]:
from keras import optimizers, applications


In [ ]:
def create_model(input_shape):
    input_tensor = Input(shape=input_shape)

    base_model = applications.ResNet152(weights='imagenet', 
                                       include_top=False,input_tensor=input_tensor)
    
    
#     base_model = applications.VGG16(weights='imagenet', 
#                                        include_top=False,input_tensor=input_tensor)

    
    
    #base_model.load_weights('../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')
    
    #x = BatchNormalization()(base_model.output)
    x = Dense(2048,activation = 'relu')(base_model.output)
    #x = Conv2D(128,(7,7), activation='relu', padding='same')(x)

    #x = GlobalAveragePooling2D()(x)#(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(600, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = GlobalAveragePooling2D()(x)
    final_output = Dense(1, activation='sigmoid', name='final_output')(x)
    model = Model(input_tensor, final_output)
    
    return model

In [ ]:
BATCH_SIZE = 10
EPOCHS = 100
WARMUP_EPOCHS = 1
LEARNING_RATE = 0.01
WARMUP_LEARNING_RATE = 1e-3
HEIGHT = 128
WIDTH = 128
CHANNEL = 3
ES_PATIENCE = 10
RLROP_PATIENCE = 3
DECAY_DROP = 0.4

In [ ]:
model = create_model(input_shape=(HEIGHT, WIDTH, CHANNEL))


In [ ]:
for layer in model.layers:
    layer.trainable = False

for i in range(-6, 0):
    model.layers[i].trainable = True


In [ ]:
metric_list = ["accuracy"]

es = EarlyStopping(monitor='val_loss', mode='min', 
                   patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', 
                          patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)

filepath="./weights.best_{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', 
                             verbose=1, save_best_only=True, mode='auto')
callback_list = [checkpoint,es,rlrop]
optimizer = optimizers.Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="binary_crossentropy",  metrics=metric_list)
# model.summary()

In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = val_generator.n//val_generator.batch_size

In [ ]:
history_finetunning = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=val_generator,
                              validation_steps=STEP_SIZE_VALID,
                              epochs=5,
                              callbacks=callback_list,
                              verbose=1).history


In [ ]:
import cv2
img = cv2.imread('./train/'+x_train.iloc[0,0])

im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(im_rgb)

In [ ]:
# modelTrained = create_model(input_shape=(HEIGHT, WIDTH, CHANNEL))
# modelTrained.load_weights("./weights.best_01-0.68.hdf5")


In [ ]:
# layer_outputs = [layer.output for layer in modelTrained.layers[1:7]]
# activation_model = Model(inputs=modelTrained.input,outputs=layer_outputs)
# activations = activation_model.predict(cv2.resize(im_rgb,(128,128)).reshape(1,128,128,3))
# layer_names = []
# for layer in modelTrained.layers[1:7]:
#     layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot
    

# images_per_row = 16
# for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
#     n_features = layer_activation.shape[-1] # Number of features in the feature map
#     size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
#     n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
#     display_grid = np.zeros((size * n_cols, images_per_row * size))
#     for col in range(n_cols): # Tiles each filter into a big horizontal grid
#         for row in range(images_per_row):
#             channel_image = layer_activation[0,
#                                              :, :,
#                                              col * images_per_row + row]
#             channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
#             channel_image /= channel_image.std()
#             channel_image *= 64
#             channel_image += 128
#             channel_image = np.clip(channel_image, 0, 255).astype('uint8')
#             display_grid[col * size : (col + 1) * size, # Displays the grid
#                          row * size : (row + 1) * size] = channel_image
#     scale = 1. / size
#     plt.figure(figsize=(scale * display_grid.shape[1],
#                         scale * display_grid.shape[0]))
#     plt.title(layer_name)
#     plt.grid(False)
#     plt.imshow(display_grid, aspect='auto', cmap='viridis')